# Lección 6: Patrones de Diseño en Python

En esta lección exploraremos tres patrones de diseño muy comunes y útiles en proyectos Python: Singleton, Factory y Strategy. Veremos su propósito, implementación y ejemplos prácticos.

## Singleton

Garantiza que una clase tenga única instancia y proporciona un punto de acceso global a ella.

In [8]:
class SingletonMeta(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            # La primera vez crea la instancia
            cls._instances[cls] = super().__call__(*args, **kwargs)
        return cls._instances[cls]
    
class Logger(metaclass = SingletonMeta):
    def __init__(self):
        self.logs = []
        
    def log(self, mensaje):
        self.logs.append(mensaje)
        
# Uso
log1 = Logger()
log2 = Logger()
log1.log("Inicio de análisis")
print(log1 is log2)     # True: misma instancia
print(log2.logs)        # ['Inicio de análisis']

True
['Inicio de análisis']


* Cuándo usarlo: cuando necesitas un único punto de configuración o registro (p.ej., conexión a base de datos, logger global).

## Factory

Provee una interfaz para crear objetos sin exponer la lógica de instanciación al cliente.

In [34]:
from abc import ABC, abstractmethod

# Productos
class Modelo(ABC):
    @abstractmethod
    def entrenar(self, datos):
        pass

class OLS(Modelo):
    def entrenar(self, datos):
        return "Ajuste OLS"
    
class Poisson(Modelo):
    def entrenar(self, datos):
        return "Ajuste Poisson"

# Fabricante (Factory)
class ModeloFactory:
    @staticmethod
    def crear_modelo(tipo, **kwargs):
        if tipo == "ols":
            return OLS(**kwargs)
        elif tipo == "poisson":
            return Poisson(**kwargs)
        else:
            raise ValueError(f"Modelo desconocido: {tipo}")
        
# Uso
m1 = ModeloFactory.crear_modelo("ols")
m2 = ModeloFactory.crear_modelo("poisson")
print(m1.entrenar(None))    # Ajuste OLS
print(m2.entrenar(None))    # Ajuste Poisson
    

Ajuste OLS
Ajuste Poisson


* Ventaja: separa la lógica de elección de implementación, facilita extensión y pruebas.

## Strategy

Define una familia de algoritmos intercambiables y los hace intercambiables dentro de un contexto común.

In [43]:
from abc import ABC, abstractmethod

# Estrategias
class EstrategiaOptim(ABC):
    @abstractmethod
    def optimizar(self, func, x0):
        pass

class GradienteDescenso(EstrategiaOptim):
    def optimizar(self, func, x0):
        return f"Optimizando {func.__name__} con Gradiente Descenso desde {x0}"
    
class NelderMead(EstrategiaOptim):
    def optimizar(self, func, x0):
        return f"Optimizando {func.__name__} con Nelder-Mead desde {x0}"

# Contexto
class Optimizador:
    def __init__(self, estrategia: EstrategiaOptim):
        self._estrategia = estrategia
        
    def set_estrategia(self, estrategia: EstrategiaOptim):
        self._estrategia = estrategia
        
    def optimizar(self, func, x0):
        return self._estrategia.optimizar(func, x0)
    
# Uso
def f_obj(x): return x**2

opt = Optimizador(GradienteDescenso())
print(opt.optimizar(f_obj, 1))
# Cambiamos de estrategia en tiempo de ejecución
opt.set_estrategia(NelderMead())
print(opt.optimizar(f_obj, 1))

Optimizando f_obj con Gradiente Descenso desde 1
Optimizando f_obj con Nelder-Mead desde 1


* Cuándo usarlo: cuando tienes varias formas de resolver un mismo problema (p.ej., distintos estimadores, métodos de optimización).